In [1]:
import Graph_Amazon_Movies as gam
import Amazon_Movie_Parser as amp
import numpy as np
from networkx.algorithms import bipartite
import networkx as nx
import Amazon_Movie_Parser as prs
import datetime
from numpy import savetxt

In [ ]:
# needed only if you don't have the movies.txt file in your local or colab directory
from google_drive_downloader import GoogleDriveDownloader as gdd
file_id1='1iytA1n2z4go3uVCwE__vIKouTKyIDjEq'
# movies.txt >> 9 GB data  ... below shared URL has the fileID to download
#https://drive.google.com/file/d/1cRLjG6Pl4JEag-utmD-7nLipzxCdmkiQ/view?usp=sharing
file_id2='1cRLjG6Pl4JEag-utmD-7nLipzxCdmkiQ'
gdd.download_file_from_google_drive(file_id=file_id2,
                                    dest_path='data/movies.txt',
                                    unzip=True)

In [2]:
start_time = datetime.datetime.now()
grp = gam.Graph_Amazon()
max_connected_gr_amazon_movies = grp.Create_Bipartite(file_name = 'data/movies.txt', \
                                                      n_movies = 2000, prs_out = 'dictionary')
end_time = datetime.datetime.now()
print('Calculation time: {}'.format(end_time-start_time))

Calculation time: 0:00:00.136464


In [3]:
bottom_nodes, top_nodes = bipartite.sets(max_connected_gr_amazon_movies)

In [4]:
# Data check
# some MOVIES don't start with B but instead full numbers like: 0790747324
print('max_connected_gr_amazon_movies:')
count_B = 0
for movie in top_nodes:
    if movie.startswith('B'):
        count_B += 1

print('Movies: {} ... starts with B: {}'.format(len(top_nodes), count_B))

count_A=0
for movie in bottom_nodes:
    if movie.startswith('A'):
        count_A += 1

print('Users: {} ... starts with A: {}'.format(len(bottom_nodes), count_A))

grp.Is_Connected_Bipartite(max_connected_gr_amazon_movies)
# edges between movies
#max_grp_top = bipartite.projected_graph(max_connected_gr_amazon_movies, grp.top_nodes)
# shows all the records BEFORE picking up the MAX CONNECTED component - most likely it will be a DISCONNECTED GRAPH
# returns a LIST
#edge_list_ALL = grp.Get_All_Edges_with_Weight(grp.gr_amazon_movies)
#print(edge_list_ALL[0:5])
#edge_list_connected = grp.Get_All_Edges_with_Weight(max_connected_gr_amazon_movies)
#list(max_connected_gr_amazon_movies.edges)[0:5]
# shows the nodes of the max connected component/subgraph
#grp.Show_Nodes()
#if your graph is too BIG, it is better not to draw it, it takes time..
#grp.Draw_Bipartite(max_connected_gr_amazon_movies)
#shows the edges with weights
#list(max_connected_gr_amazon_movies.edges.data('weight', default=1))[0:5]

max_connected_gr_amazon_movies:
Movies: 20 ... starts with B: 15
Users: 1832 ... starts with A: 1832
Is connected: True ... Is bipartite: True


In [5]:
#saves the movies/graph in the format of:
'''
A141HP4LYPW,B003AI2VGA,3.0
A141HP4LYPW,B000063W1R,4.0
A141HP4LYPW,6304286961,5.0
'''
#max_connected_gr_amazon_movies
file_name_saved = amp.Save_Movies(max_connected_gr_amazon_movies)

Graph info saved in: Movies_Connected_1885_11.5.2020_21.57.36.txt


In [6]:
# returns a list in the format of:
# [['userId', 'productId', 'score'], ['A141HP4LYPW', 'B003AI2VGA', '3.0']]
#the_giant_component_read = amp.Read_Connected_Movies("Movies_Connected_4643_11.5.2020_21.11.17.txt")
the_giant_component_read = amp.Read_Connected_Movies(file_name_saved)
#test[0].split(sep=",")

In [7]:
# it is mostly for a double check to see if the movies were saved to the file properly 
#and check if we read them and create a graph it will still be a connected graph or not
grp_giant = grp.Create_Graph_From_List_WITH_Weight(the_giant_component_read)

In [8]:
# double check if the numbers are matching before saving the records
# and after creating the graph from the saved file
print('BEFORE: # of nodes: {} ... # of edges: {}'.format(len(max_connected_gr_amazon_movies.nodes), len(max_connected_gr_amazon_movies.edges)))
print('AFTER: # of nodes: {} ... # of edges: {}'.format(len(grp_giant.nodes), len(grp_giant.edges)))

BEFORE: # of nodes: 1852 ... # of edges: 1885
AFTER: # of nodes: 1852 ... # of edges: 1885


In [9]:
# here is the check of the graph read
grp.Is_Connected_Bipartite(grp_giant)

Is connected: True ... Is bipartite: True


In [10]:
# this is to fetch all nodes without edges, so that we can use them to predict the edges
grp_giant_no_edges = grp.Create_Graph_From_List_NO_EDGE(grp_giant)
#if you check it, you'll see no edges but nodes
#grp_giant_no_edges.edges
#list(grp_giant_no_weights.nodes)[0:5]
#grp.Is_Connected_Bipartite(grp_giant_no_weights)

In [ ]:
start_time = datetime.datetime.now()
df_movies = amp.Reorganize_Edges_DataFrame(max_connected_gr_amazon_movies, input_type = 'Graph')

node_movies = df_movies['movieId'].unique()
node_users = df_movies['userId'].unique()
nodes = np.concatenate((node_users, node_movies), axis=0)
n_nodes = len(nodes)
#n_nodes = len(max_connected_gr_amazon_movies.nodes)
print('# of nodes in the giant connected component: {}'.format(n_nodes))
# Probability matrix, if there is an edge between node[i] and node[j], 
# in other words, if a user reviewed a movie with a score from 1 to 5, P[i][j]=score[ij]
# then the row will be normalized to have the values between 0 and 1
#the weight will be the

P = np.zeros((n_nodes, n_nodes))
R = np.zeros((n_nodes, n_nodes))

i = 0
for n_r in nodes:
    j = 0
    for n_c in nodes:
        edge_index = df_movies.query("movieId == '" + n_r + "' and userId == '" + n_c + "'").index
        if len(edge_index) != 0:
            P[i][j] = df_movies.iloc[edge_index[0], 2]
            P[j][i] = P[i][j]
            #print('i={} .. j={} .. n_r={} .. n_c={}'.format(i, j, n_r, n_c))
        j += 1
    print('node-{}: {} is checked out of {} nodes.'.format(i, n_r, n_nodes))
    i += 1
    
end_time = datetime.datetime.now()
print('Calculation time: {}'.format(end_time-start_time))

Graph has been reorganized as a DataFrame, row format: ['userId', 'movieId', 'score']
# of nodes in the giant connected component: 1852
node-0: B003AI2VGA is checked out of 1852 nodes.
node-1: B000063W1R is checked out of 1852 nodes.
node-2: B0001G6PZC is checked out of 1852 nodes.
node-3: B0016OLXN2 is checked out of 1852 nodes.
node-4: B00004CQT3 is checked out of 1852 nodes.
node-5: B002OHDRF2 is checked out of 1852 nodes.
node-6: B0078V2LCY is checked out of 1852 nodes.
node-7: B0092MDYV4 is checked out of 1852 nodes.
node-8: B000CDSS86 is checked out of 1852 nodes.
node-9: 0800103688 is checked out of 1852 nodes.
node-10: 0790747324 is checked out of 1852 nodes.
node-11: B000NDFLWG is checked out of 1852 nodes.
node-12: B008FPU7AA is checked out of 1852 nodes.
node-13: 6300157423 is checked out of 1852 nodes.
node-14: B00004CQTF is checked out of 1852 nodes.
node-15: B000IZJR3I is checked out of 1852 nodes.
node-16: 630436119X is checked out of 1852 nodes.
node-17: B001GBPZRU is c

node-158: A3SPJY549DX is checked out of 1852 nodes.
node-159: A2RW7HXNEJ0 is checked out of 1852 nodes.
node-160: A4OWXQLYVER is checked out of 1852 nodes.
node-161: AG9A3ZL7ODL is checked out of 1852 nodes.
node-162: A2XTMDDO0UC is checked out of 1852 nodes.
node-163: A3GV24GH1AF is checked out of 1852 nodes.
node-164: A1VT9SADFD9 is checked out of 1852 nodes.
node-165: A31VEVBN7K8 is checked out of 1852 nodes.
node-166: A1O05HYBO2L is checked out of 1852 nodes.
node-167: A1UMSB7LAW0 is checked out of 1852 nodes.
node-168: A22QDGL361U is checked out of 1852 nodes.
node-169: A27GP310AJR is checked out of 1852 nodes.
node-170: A39CX0EE4BZ is checked out of 1852 nodes.
node-171: A30PZUE2YQH is checked out of 1852 nodes.
node-172: A91VKBOACYQ is checked out of 1852 nodes.
node-173: A3VC1CIRFDX is checked out of 1852 nodes.
node-174: A2PKRPEODLQ is checked out of 1852 nodes.
node-175: A3A4WQL80WO is checked out of 1852 nodes.
node-176: A273SGLDMWW is checked out of 1852 nodes.
node-177: A1

node-316: A1KBEHXGW71 is checked out of 1852 nodes.
node-317: A3UP5OPI1HA is checked out of 1852 nodes.
node-318: A36TASDZP9B is checked out of 1852 nodes.
node-319: A3J510E9W1C is checked out of 1852 nodes.
node-320: A1GVLNWPIGL is checked out of 1852 nodes.
node-321: AON6QJ88F2A is checked out of 1852 nodes.
node-322: A2VLH0R0UP9 is checked out of 1852 nodes.
node-323: ASUEUK3RMFZ is checked out of 1852 nodes.
node-324: A2NK3PX0BLE is checked out of 1852 nodes.
node-325: A2ZYB2S79GU is checked out of 1852 nodes.
node-326: A2D4RUMDYEF is checked out of 1852 nodes.
node-327: A2AW4A1E87R is checked out of 1852 nodes.
node-328: AQ01Q3070LT is checked out of 1852 nodes.
node-329: A2YWGP8UUU6 is checked out of 1852 nodes.
node-330: A1HBXCEAW5F is checked out of 1852 nodes.
node-331: A2AFM0SLLFP is checked out of 1852 nodes.
node-332: A2887J6P8VP is checked out of 1852 nodes.
node-333: A3I3UF46UWQ is checked out of 1852 nodes.
node-334: A2FRKEXDXDN is checked out of 1852 nodes.
node-335: A2

node-474: A2H8EF6UQIK is checked out of 1852 nodes.
node-475: A20EEWWSFMZ is checked out of 1852 nodes.
node-476: A15N5W09G9K is checked out of 1852 nodes.
node-477: AIOXMENE1HB is checked out of 1852 nodes.
node-478: A3160YSU1KR is checked out of 1852 nodes.
node-479: A2RRCZLVCA7 is checked out of 1852 nodes.
node-480: A3MCMOX8M5E is checked out of 1852 nodes.
node-481: A1ZLGSW21GS is checked out of 1852 nodes.
node-482: A2K1V1Z41X9 is checked out of 1852 nodes.
node-483: A11XQIT4N5T is checked out of 1852 nodes.
node-484: A2ZZN2ADSY4 is checked out of 1852 nodes.
node-485: A3JT29L4YFE is checked out of 1852 nodes.
node-486: A7EUWJK1EW2 is checked out of 1852 nodes.
node-487: A110UAX71H8 is checked out of 1852 nodes.
node-488: A1RRT0R0ARC is checked out of 1852 nodes.
node-489: A231SN9GIOZ is checked out of 1852 nodes.
node-490: A22U5T9Q0GI is checked out of 1852 nodes.
node-491: A36C2A4SWAT is checked out of 1852 nodes.
node-492: A5HQ4DKZX91 is checked out of 1852 nodes.
node-493: A1

node-632: AI45MQMDG3F is checked out of 1852 nodes.
node-633: AV3RUX4IMND is checked out of 1852 nodes.
node-634: AG5XM49AZVI is checked out of 1852 nodes.
node-635: A12X8OWP4N7 is checked out of 1852 nodes.
node-636: A2XRMQA6PJ5 is checked out of 1852 nodes.
node-637: ANS2ERV3VRX is checked out of 1852 nodes.
node-638: A2J9L2AWKTG is checked out of 1852 nodes.
node-639: A37ZXTO6BFL is checked out of 1852 nodes.
node-640: A3I110RYL8Q is checked out of 1852 nodes.
node-641: AVTA38OJEY7 is checked out of 1852 nodes.
node-642: A3F3MVVYJTD is checked out of 1852 nodes.
node-643: AOJ0ORN9VTP is checked out of 1852 nodes.
node-644: A27DRF925AA is checked out of 1852 nodes.
node-645: AS7BQSB2PLV is checked out of 1852 nodes.
node-646: A33IRCG12HK is checked out of 1852 nodes.
node-647: A13AFOYULBG is checked out of 1852 nodes.
node-648: A395EVWBUDT is checked out of 1852 nodes.
node-649: A324507DQU0 is checked out of 1852 nodes.
node-650: A16KRCLLTHS is checked out of 1852 nodes.
node-651: A1

...

node-948: A2TQW5QAGK5 is checked out of 1852 nodes.
node-949: A4JP4T1F95D is checked out of 1852 nodes.
node-950: A11EUXRL9HX is checked out of 1852 nodes.
node-951: A157HLB7Z0J is checked out of 1852 nodes.
node-952: A3OXJB66H8L is checked out of 1852 nodes.
node-953: A2T7TZBOTP6 is checked out of 1852 nodes.
node-954: A258MSMKY27 is checked out of 1852 nodes.
node-955: A1IBZ78E075 is checked out of 1852 nodes.
node-956: A1S0RU3DSFD is checked out of 1852 nodes.
node-957: A7GSGC0G8R6 is checked out of 1852 nodes.
node-958: AQDQVC3SBCO is checked out of 1852 nodes.
node-959: A3TEH90X39W is checked out of 1852 nodes.
node-960: AP6FB28W66L is checked out of 1852 nodes.
node-961: A1YQ6QB2127 is checked out of 1852 nodes.
node-962: A3MJMDJ8N9S is checked out of 1852 nodes.
node-963: A2I0D4U84MC is checked out of 1852 nodes.
node-964: AHK6FJP078E is checked out of 1852 nodes.
node-965: A2G0WJZ9Q7Y is checked out of 1852 nodes.
node-966: A3LZBOBV9H1 is checked out of 1852 nodes.
node-967: A2

node-1104: A1ST7J8IFUG is checked out of 1852 nodes.
node-1105: AHF97M45JFC is checked out of 1852 nodes.
node-1106: AOXX0U2T2BG is checked out of 1852 nodes.
node-1107: A2KHKJDDV7M is checked out of 1852 nodes.
node-1108: A2STSWXXID2 is checked out of 1852 nodes.
node-1109: A1FOLVC0BVX is checked out of 1852 nodes.
node-1110: AFXTKAO0CB3 is checked out of 1852 nodes.
node-1111: A1A5SGSGIHV is checked out of 1852 nodes.
node-1112: A3U7NO0G7JZ is checked out of 1852 nodes.
node-1113: ACM2487CXB9 is checked out of 1852 nodes.
node-1114: A3QO324Z4CU is checked out of 1852 nodes.
node-1115: APYWZNEO6CI is checked out of 1852 nodes.
node-1116: A3Q89MR4M56 is checked out of 1852 nodes.
node-1117: A24DS9QM4BM is checked out of 1852 nodes.
node-1118: A3OSXG8BXIP is checked out of 1852 nodes.
node-1119: A398FSYYCHS is checked out of 1852 nodes.
node-1120: A3PB934FJFZ is checked out of 1852 nodes.
node-1121: A273J78OJYN is checked out of 1852 nodes.
node-1122: A3SV3W434LP is checked out of 1852 

node-1259: A26YH615RE7 is checked out of 1852 nodes.
node-1260: ACCBJXX6ZJL is checked out of 1852 nodes.
node-1261: A3CTD32536A is checked out of 1852 nodes.
node-1262: A1CZ8JTF1JC is checked out of 1852 nodes.
node-1263: A3RV4XKMA6B is checked out of 1852 nodes.
node-1264: A3CL82UITQJ is checked out of 1852 nodes.
node-1265: A2PWAPST6JH is checked out of 1852 nodes.
node-1266: A7WP90L9QG2 is checked out of 1852 nodes.
node-1267: AP5V6ESC7K4 is checked out of 1852 nodes.
node-1268: A3M6X99R341 is checked out of 1852 nodes.
node-1269: A258EDM8MAZ is checked out of 1852 nodes.
node-1270: AP3B615GM19 is checked out of 1852 nodes.
node-1271: A3F3T8UMTHR is checked out of 1852 nodes.
node-1272: ANPKULUIW0V is checked out of 1852 nodes.
node-1273: A33KZ2CS6WE is checked out of 1852 nodes.
node-1274: AZ7SL665RJF is checked out of 1852 nodes.
node-1275: A1IYZS46NB2 is checked out of 1852 nodes.
node-1276: AYQ9OC2D8HP is checked out of 1852 nodes.
node-1277: A34QL4FP5FH is checked out of 1852 

node-1414: A35UVDJTA26 is checked out of 1852 nodes.
node-1415: A133CUOCQTP is checked out of 1852 nodes.
node-1416: AYWSFRCIMOA is checked out of 1852 nodes.
node-1417: A3PDJ5XPAKP is checked out of 1852 nodes.
node-1418: A29S1MTFU2D is checked out of 1852 nodes.
node-1419: A36K2N527TX is checked out of 1852 nodes.
node-1420: A2DDP69VU2T is checked out of 1852 nodes.
node-1421: A2M8I0UNER3 is checked out of 1852 nodes.
node-1422: A3KT2RQJ1LP is checked out of 1852 nodes.
node-1423: A82R4WKX62L is checked out of 1852 nodes.
node-1424: A2341EETXMR is checked out of 1852 nodes.
node-1425: A1D88GB7EBF is checked out of 1852 nodes.
node-1426: A1NLJT22OOP is checked out of 1852 nodes.
node-1427: A12XRMMPDPU is checked out of 1852 nodes.
node-1428: A14T4RYJH1C is checked out of 1852 nodes.
node-1429: A3E975KR2HR is checked out of 1852 nodes.
node-1430: AOIFNL9ZHM5 is checked out of 1852 nodes.
node-1431: AFMUUJT7J32 is checked out of 1852 nodes.
node-1432: A31X4DOV3VZ is checked out of 1852 

node-1569: A3K0E9JKGAH is checked out of 1852 nodes.
node-1570: A369XPXR47N is checked out of 1852 nodes.
node-1571: A38GAOW7P61 is checked out of 1852 nodes.
node-1572: A23IMHDBG60 is checked out of 1852 nodes.
node-1573: A3EKQV7J4GE is checked out of 1852 nodes.
node-1574: A3FEQXNCFC6 is checked out of 1852 nodes.
node-1575: A1KP0IVWU0R is checked out of 1852 nodes.
node-1576: A3I78A6XPF1 is checked out of 1852 nodes.
node-1577: A3VV4GUGLUV is checked out of 1852 nodes.
node-1578: A157GIR4YJX is checked out of 1852 nodes.
node-1579: AJIP72W023R is checked out of 1852 nodes.
node-1580: A3KX3CTPV9U is checked out of 1852 nodes.
node-1581: A3JJUW47D3A is checked out of 1852 nodes.
node-1582: A1T0OJYXT2B is checked out of 1852 nodes.
node-1583: A3L7XKLTP12 is checked out of 1852 nodes.
node-1584: A2F96GST4QD is checked out of 1852 nodes.
node-1585: A1YB29BIDUL is checked out of 1852 nodes.
node-1586: A2PV6GK1HV5 is checked out of 1852 nodes.
node-1587: A279AU7MXJ7 is checked out of 1852 

In [ ]:
def Normalize_Array(arr_to_normalize):
    arr_sum_per_row = arr_to_normalize.sum(axis=1)
    arr_norm = arr_to_normalize/np.tile(arr_sum_per_row, (arr_sum_per_row.shape[0], 1)).T
    return arr_norm

In [ ]:
# until roun(, 3) >> it's ok. when we round to 4, then it messes up..
# even with round(3) >> if we save in a file, it messes up..
P_norm = np.round(Normalize_Array(P), 3)
#P_norm = Normalize_Array(P)

In [ ]:
P_norm

In [ ]:
# SUM-check
#print('SUM-check')
#P_norm.sum(axis=1)

In [ ]:
savetxt(amp.FileNameUnique(prefix = "P_norm_", suffix = '.csv'), P_norm, delimiter=',')